In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_path = '/content/drive/MyDrive/모듈프로젝트3_2조/허현준/dataset/train_V2.csv'

In [ ]:
train = pd.read_csv(train_path)

## 1. matchType 속성값 단일화

In [ ]:
# matchType : solo, duo, squad 3종류 단일화
'''
solo  <-- solo,solo-fpp,normal-solo,normal-solo-fpp
duo   <-- duo,duo-fpp,normal-duo,normal-duo-fpp,crashfpp,crashtpp
squad <-- squad,squad-fpp,normal-squad,normal-squad-fpp,flarefpp,flaretpp
'''

# 함수선언
mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) or ('crash' in x) else 'squad'

# 변경
train['matchType'] = train['matchType'].apply(mapper)

## 2. 결측지 제거

In [ ]:
# 결측지 확인
train.isnull().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64

In [ ]:
# 결측지 제거하고 확인
train.dropna(subset=['winPlacePerc'], inplace = True)
train.isnull().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

## 3. IQR 방식으로 이상치 찾기
- 극상위, 극하위 유저 찾는 용도

In [ ]:
# 이상치 찾기
# 이상치 제거, 리턴은 DataFrame
def find_outlier(data_col):
  q1, q3 = np.percentile(data_col,[0.01, 99.99])
  iqr = q3 - q1
  lower = q1 - (iqr * 1.5)
  upper = q3 + (iqr * 1.5)

  # 이상치 데이터
  outer_df = train[(data_col > upper) | (data_col < lower)]

  # 이상치를 제외한 데이터
  inner_df = train[(upper >= data_col) & (data_col >= lower)]
  
  return outer_df, inner_df


## 4. 데미지가 0인데 기절시키거나 킬한 데이터들

- damageDealt : 가한 데미지의 총합. 스스로 가한 데미지는 제외.
- DBNOs : 기절시킨 적의 수

In [ ]:
# 데미지가 0인데 죽이거나 기절시킨 데이터 제거
# 리턴값은 제거된 데이터프레임

def rm_no_damage(data):
  print("이상치  제거 할 DataFrame shape : ", data.shape)
  no_damage = data[(data["damageDealt"] ==0 ) & ((data["kills"] > 0) | (data['DBNOs'] > 0))]

  idx = no_damage.index
  print("이상치 데이터 수 :",len(idx))

  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  
  return df

## 5. 1등이 존재하지 않는 매치 

- winPlacePerc : 백분위수에 기반한 타겟의 예측. 1은 1등을 의미하며 0은 마지막 등수를 의미한다.

- 매치별로 그룹화하여 winPlacePerc의 max값을 보면 무조건 1이여야한다.(1등이 존재하지 않는 매치는 오류니까)

In [ ]:
# 1등이 존재하지 않는 매치
def no_winner(data):
  # 의미 -> 매치별로 그룹화해서 'winPlacePerc' max 값 확인 
  # 결과 1이 아니다 -> 해당 매치에 1등이 없다 -> 해당 매치는 오류난 매치
  maxPlacePerc = data.groupby('matchId')['winPlacePerc'].max()
  df = pd.DataFrame(maxPlacePerc[maxPlacePerc != 1]).reset_index()
  print("이상치  제거 할 DataFrame shape : ", data.shape)

  # 해당 'matchId'가 포함된 데이터프레임 확인
  ol = df['matchId']
  ex = data[data['matchId'].isin(ol)]

  # 해당 index 추출
  idx = ex.index
  print("이상치 데이터 수 :",len(idx))

  # 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  return df

## 6. 실질적 플레이어(numgroup)이 1명인 매치 

- 'maxPlace' > 1 : 해당 매치에 플레이어 수 는 1명보다 많다
- 'numgroups' == 1 : 해당 매치에서 실질적으로 플레이한 사람(그룹)의 수는 1명 
- 100명이서 시험보는데 나머지를 모두 수면제로 재우고 혼자 시험봐서 1등한것과 유사 
- 또는 팀별 프로젝트를 하는데 반 전체가 한 팀이라 1등이자 꼴찌임

In [ ]:
# 'numGroups' 매커니즘은 아마 winPlacePerc가 같은 사람들을 그룹화해서 그 숫자를 센듯
def rm_no_player(data):
  print("이상치  제거 할 DataFrame shape : ", data.shape)

  # 'maxPlace' > 1  : 튕긴 사람을 포함한 플레이어 수는 1명보다 많음 
  # 'numGroups' == 1 : 실질적으로 플레이한 그룹(사람)은 혼자임 -> 혼자 게임
  no_player = data[(data['maxPlace'] > 1) & (data['numGroups'] == 1)]
  
  # 해당 데이터 인덱스추출
  idx = no_player.index
  print("이상치 데이터 수 :",len(idx))

  # 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  
  return df

## 7. 듀오,스쿼드별 그룹인원수 안 맞는 경우

In [ ]:
def people_num_not_matchtype(data):
  print("이상치  제거 할 DataFrame shape : ", data.shape)

  # 'matchId', 'groupId', 'matchType'을 그룹하여 ID를 카운터해서 'players' 컬럼 추가
  group = data.groupby(['matchId','groupId','matchType'])['Id'].count().to_frame('players').reset_index()

  # 'matchType'과 플레이어 수가 일치하지 않는 경우 확인
  solo_ex = group[(group['matchType'] == 'solo') & (group['players'] != 1)]
  duo_ex = group[(group['matchType'] == 'duo') & (group['players'] > 2)]
  squad_ex = group[(group['matchType'] == 'squad') & (group['players'] > 4)]


  # 매치당 비정상적인 그룹이 포함된  숫자 정보 가진 'Abnormal' 컬럼 생성후 인덱스 초기화
  solo_ex = solo_ex.groupby(['matchId'])['players'].count().to_frame('Abnormal').reset_index()
  duo_ex = duo_ex.groupby(['matchId'])['players'].count().to_frame('Abnormal').reset_index()
  squad_ex = squad_ex.groupby(['matchId'])['players'].count().to_frame('Abnormal').reset_index()


  # 모든 매치에 비이상적인 그룹수를 가진 팀이 존재함
  # 따라서 비이상적인 그룹이 4그룹보다 많으면 매치가 과도하게 오염되었다고 설정
  solo_ex = solo_ex[solo_ex['Abnormal'] > 1]
  duo_ex = duo_ex[duo_ex['Abnormal'] > 1]
  squad_ex = squad_ex[squad_ex['Abnormal'] > 1]

  # 매치 아이디를 리스트로 저장 
  ol_solo = list(solo_ex['matchId'].unique())
  ol_duo = list(duo_ex['matchId'].unique())
  ol_squad = list(squad_ex['matchId'].unique())
  print("비이상적인 그룹이 4그룹보다 많은 솔로 매치 타입 수", len(ol_solo))
  print("비이상적인 그룹이 4그룹보다 많은 듀오 매치 타입 수", len(ol_duo))
  print("비이상적인 그룹이 4그룹보다 많은 스쿼드 매치 타입 수",len(ol_squad))


  # 각 매치별 오염된 매치 통합후 제거
  # 이제까지 구한 모든 리스트를 합쳐서 통합한다.
  ol_total = ol_solo + ol_duo + ol_squad
  print("모든 매치에 비이상적인 그룹이 4그룹보다 많은 경우 수",len(ol_total))

  # 비정상적인 'matchId'가 포함된 데이터프레임
  ex = data[data['matchId'].isin(ol_total)]
  
  # 인덱스 저장
  idx = ex.index
  print("제거할 이상치 수",len(idx))

  # 인덱스로 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  return df

## 8. 매치 킬 수 가 이상한 경우

- 배그의 경우 한 매치에 최대 100명이 들어가기 때문에 매치당 킬 수의 합은 최대 99이다

- 그렇지 않은 경우는 오류가 난 데이터 이므로 이를 색출한다.


In [ ]:
def odd_mathch_kill(data):

  print("이상치  제거 할 DataFrame shape : ", data.shape)
  grp = train8.groupby('matchId',as_index = False).sum()

  # 매치별 킬 수 합이 99가 넘는 데이터들을 뽑는다.
  ol = grp[grp['kills'] > 99]['matchId']

  # train에서 ol 에 존재하는 매치에 해당하는 데이터들을 모두 본다.
  ex = data[data['matchId'].isin(ol)]

  # 인덱스 추출
  idx = ex.index
  print("이상치 데이터 수 :",len(idx))

  # 이상치 제거후 인덱스 재정렬
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  
  return df

## 9. 매치내 사람 수 보다 킬 수가 더 많은 경우 

- 배그의 경우 한 매치에 참여한 사람수는 maxPlace -> 킬 수가 이것보다 같거나 크면 말이 안됨 




In [ ]:
def overkill_people_match(data):
  print("이상치  제거 할 DataFrame shape : ", data.shape)
  # 매치 내
  ex = data[data['kills'] >= data['maxPlace']]['matchId'].unique()

  ol = list(ex)

  # isin함수를 사용 
  # train에서 ol에 존재하는 매치에 해당하는 데이터들을 모두 본다.
  ex = data[data['matchId'].isin(ol)]


  # 인덱스 정보를 idx 변수에 저장한다.
  idx = ex.index
  print("이상치 데이터 수 :",len(idx))

  # ex에 해당하는 인덱스와 일치하는 인덱스를 가진 데이터를 전부 날린다
  # 원본을 건드리지 말고 임시로 temp에 넣는다.
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  return df

## 10. 운전한거리에 비해 로드킬수가 과하게 많음

- rideDistance : 미터로 측정한 차량의 총 이동 거리입니다.

In [ ]:
# 운전별로안하고 로드킬 비정상적으로 많이한 놈 잡기

# 운전거리 최소 1미터에 1킬한다고 가정
def roadkill_bug(data):

  print("이상치  제거 할 DataFrame shape : ", data.shape)
  out_roadkill = find_outlier(train['roadKills'])[0]

  roadkill_num = out_roadkill['roadKills'].unique()
  print("로드킬 상위 0.01% 킬 수", roadkill_num)

  road_bug = data[(data['rideDistance']* 100 < roadkill_num.max()) &  (data['roadKills'] > roadkill_num.min())]
  idx = road_bug.index
  print("이상치 데이터 수 :",len(idx))

  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)

  return df

## 12. 무기 사용이 0인데 1등을 한 경우의 사람
-  무기를 사용하지 않고 1등을 할 수 있는 경우(duo, squd)
- 무기를 사용하지 않고  'solo'가 1등을 할 수 있나? - > 없음

In [ ]:
def no_weapon_solo(data):
  
  print("이상치  제거 할 DataFrame shape : ", data.shape)
  
  # 1등인데, 무기 획득 = 0 이고 매치 타입이 'solo'인 경우 
  no_weapon_solo_df = data[((train['winPlacePerc'] > 0.99) & (data['weaponsAcquired'] == 0) & (data['matchType'] == "solo"))]
  
  # 위 경우의 인덱스 추출
  idx = no_weapon_solo_df.index
  print("이상치 데이터 수 :",len(idx))

  # 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  return df

In [ ]:
train3 = rm_no_damage(train)

이상치  제거 할 DataFrame shape :  (4446965, 29)
이상치 데이터 수 : 22737
이상치 제거 후 DataFrame shape :  (4424228, 29)


In [ ]:
train4 = no_winner(train3)

이상치  제거 할 DataFrame shape :  (4424228, 29)
이상치 데이터 수 : 3509
이상치 제거 후 DataFrame shape :  (4420719, 29)


In [ ]:
train5 =  rm_no_player(train4)

이상치  제거 할 DataFrame shape :  (4420719, 29)
이상치 데이터 수 : 0
이상치 제거 후 DataFrame shape :  (4420719, 29)


In [ ]:
train8 = people_num_not_matchtype(train5)

이상치  제거 할 DataFrame shape :  (4420719, 29)
비이상적인 그룹이 4그룹보다 많은 솔로 매치 타입 수 5152
비이상적인 그룹이 4그룹보다 많은 듀오 매치 타입 수 12507
비이상적인 그룹이 4그룹보다 많은 스쿼드 매치 타입 수 24980
모든 매치에 비이상적인 그룹이 4그룹보다 많은 경우 수 42639
제거할 이상치 수 3966339
이상치 제거 후 DataFrame shape :  (454380, 29)


In [ ]:
train9 = odd_mathch_kill(train8)

이상치  제거 할 DataFrame shape :  (454380, 29)
이상치 데이터 수 : 641
이상치 제거 후 DataFrame shape :  (453739, 29)


In [ ]:
train10 = overkill_people_match(train9)

이상치  제거 할 DataFrame shape :  (453739, 29)
이상치 데이터 수 : 476
이상치 제거 후 DataFrame shape :  (453263, 29)


In [ ]:
train11 = roadkill_bug(train10)

이상치  제거 할 DataFrame shape :  (453263, 29)
로드킬 상위 0.01% 킬 수 [ 8  9 14 11 18 10]
이상치 데이터 수 : 0
이상치 제거 후 DataFrame shape :  (453263, 29)


In [ ]:
train12 = no_weapon_solo(train11)

이상치  제거 할 DataFrame shape :  (453263, 29)
이상치 데이터 수 : 379
이상치 제거 후 DataFrame shape :  (452884, 29)


In [ ]:
train12.to_csv('/content/drive/MyDrive/모듈프로젝트3_2조/허현준/dataset/train_V3.csv', index = False)

In [ ]:
# path_2 = '/content/drive/MyDrive/모듈프로젝트3_2조/허현준/dataset/train_V3.csv'

In [ ]:
# df = pd.read_csv(path_2)

In [ ]:
# df